In [1]:
import pandas as pd
import numpy as np

import bambi as bmb


# define model data
data = pd.DataFrame(
    {
        "y": np.random.normal(size=50),
        "g": np.random.choice(["Yes", "No"], size=50),
        "x1": np.random.normal(size=50),
        "x2": np.random.normal(size=50),
    }
)

/Users/yannmclatchie/Desktop/kulprit/.venv/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# define and fit model with MCMC
model = bmb.Model("y ~ x1 + x2", data, family="gaussian")
idata = model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [y_sigma, x2, x1, Intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 33 seconds.


In [3]:
chain_n = len(idata.posterior.coords.get("chain"))
draw_n = len(idata.posterior.coords.get("draw"))

In [4]:
x1 = idata.posterior["x1"]
x1_stacked = x1.stack(samples=("chain", "draw"))
x1_stacked

<xarray.DataArray 'x1' (samples: 4000)>
array([-0.38035293, -0.44420227, -0.44756661, ..., -0.29704301,
       -0.24961134, -0.49611936])
Coordinates:
  * samples  (samples) MultiIndex
  - chain    (samples) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3
  - draw     (samples) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999

Have a look at the MultiIndex above. The order we used for the stacking is `("chain", "draw")`. It is like saying

In [5]:
samples = [None] * chain_n * draw_n
for i in range(chain_n):
    for j in range(draw_n):
        idx = i * draw_n + j
        samples[idx] = (i, j)

In [6]:
print(samples[:4])
print(samples[-4:])

[(0, 0), (0, 1), (0, 2), (0, 3)]
[(3, 996), (3, 997), (3, 998), (3, 999)]


In terms of [NumPy documentation](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) it is using `order="C"` because the last axis index changes fastest.

If you take the values out of that `xarray.DataArray`, you need to make sure that `.stack()` and `.reshape()` are performing the operations in the same consistent manner.

In this case, they both use `order="C"` and that's why the following recovers the original values.

In [7]:
print(x1_stacked.shape)
x1_stacked.values

(2000,)


array([0.09471466, 0.03987166, 0.1564112 , ..., 0.11219181, 0.14391421,
       0.02960541])

In [8]:
x1_stacked.values.reshape((chain_n, draw_n))

array([[ 0.09471466,  0.03987166,  0.1564112 , ...,  0.15913814,
         0.15913814,  0.09562304],
       [-0.06398758,  0.27575427, -0.11687804, ...,  0.11219181,
         0.14391421,  0.02960541]])

In [9]:
x1_stacked.values.reshape((chain_n, draw_n)) == x1.values

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In summary, it would be good to test that you're effectively applying a function and its inverse (in this case, `.stack()` and `.reshape()`).

If `.reshape()` worked in a different way, it would not be its inverse.

One case where you need to pay attention to the order of the dims, is when you do `.stack()` on only a subset of the dimensions. Let's see the following example

In [10]:
y = idata.log_likelihood["y"]
y

<xarray.DataArray 'y' (chain: 2, draw: 1000, y_dim_0: 50)>
array([[[-1.18700312, -3.40351743, -1.19118703, ..., -1.86838958,
         -1.28942953, -2.96891932],
        [-1.07269478, -3.93967493, -1.08027563, ..., -1.99244375,
         -1.22481654, -3.25304538],
        [-1.017506  , -4.29964893, -1.00147009, ..., -2.37839827,
         -1.26328855, -3.03007607],
        ...,
        [-0.90754508, -4.65439194, -0.91177367, ..., -1.95661394,
         -1.03851242, -4.20858918],
        [-0.90754508, -4.65439194, -0.91177367, ..., -1.95661394,
         -1.03851242, -4.20858918],
        [-1.03278457, -3.61557726, -1.04164816, ..., -2.14648094,
         -1.19335731, -3.19553823]],

       [[-1.01900777, -3.55261817, -1.07891603, ..., -2.21707611,
         -1.19306125, -3.28668189],
        [-1.10520991, -3.67099903, -1.10815681, ..., -1.92014054,
         -1.22191428, -3.07651213],
        [-0.8875274 , -3.9380779 , -0.98656634, ..., -2.72738148,
         -1.17188421, -3.54247474],
        ...,
        [-0.94304335, -3.96867745, -0.95160923, ..., -2.29231812,
         -1.13431909, -3.49779756],
        [-1.01580124, -4.58236042, -1.01384167, ..., -1.61531873,
         -1.07789365, -4.19952126],
        [-1.19479568, -3.17273136, -1.18029229, ..., -2.33599077,
         -1.40103488, -2.44007612]]])
Coordinates:
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
  * y_dim_0  (y_dim_0) int64 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49

In [11]:
y_stacked = y.stack(samples=("chain", "draw"))
y_stacked


<xarray.DataArray 'y' (y_dim_0: 50, samples: 2000)>
array([[-1.18700312, -1.07269478, -1.017506  , ..., -0.94304335,
        -1.01580124, -1.19479568],
       [-3.40351743, -3.93967493, -4.29964893, ..., -3.96867745,
        -4.58236042, -3.17273136],
       [-1.19118703, -1.08027563, -1.00147009, ..., -0.95160923,
        -1.01384167, -1.18029229],
       ...,
       [-1.86838958, -1.99244375, -2.37839827, ..., -2.29231812,
        -1.61531873, -2.33599077],
       [-1.28942953, -1.22481654, -1.26328855, ..., -1.13431909,
        -1.07789365, -1.40103488],
       [-2.96891932, -3.25304538, -3.03007607, ..., -3.49779756,
        -4.19952126, -2.44007612]])
Coordinates:
  * y_dim_0  (y_dim_0) int64 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49
  * samples  (samples) MultiIndex
  - chain    (samples) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
  - draw     (samples) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999

Note the `samples` dim was sent to the tail of the dimensions. Let's see the following comparison.

In [12]:
y_stacked.values.reshape((chain_n, draw_n, 50)) == y.values

array([[[ True, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False,  True]]])

All `False`! That's because `y_stacked` is of shape `(50, 2000)`. We can fix it in several ways

Something that seems to work, but it's not very clear

In [13]:
y_stacked.values.T.reshape((chain_n, draw_n, 50)) == y.values

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]])

A better solution: transpose being explicit on the dimensions.

In [14]:
y_stacked = y.stack(samples=("chain", "draw")).transpose("samples", "y_dim_0")
y_stacked


<xarray.DataArray 'y' (samples: 2000, y_dim_0: 50)>
array([[-1.18700312, -3.40351743, -1.19118703, ..., -1.86838958,
        -1.28942953, -2.96891932],
       [-1.07269478, -3.93967493, -1.08027563, ..., -1.99244375,
        -1.22481654, -3.25304538],
       [-1.017506  , -4.29964893, -1.00147009, ..., -2.37839827,
        -1.26328855, -3.03007607],
       ...,
       [-0.94304335, -3.96867745, -0.95160923, ..., -2.29231812,
        -1.13431909, -3.49779756],
       [-1.01580124, -4.58236042, -1.01384167, ..., -1.61531873,
        -1.07789365, -4.19952126],
       [-1.19479568, -3.17273136, -1.18029229, ..., -2.33599077,
        -1.40103488, -2.44007612]])
Coordinates:
  * y_dim_0  (y_dim_0) int64 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49
  * samples  (samples) MultiIndex
  - chain    (samples) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
  - draw     (samples) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999

In [15]:
y_stacked.values.reshape((chain_n, draw_n, 50)) == y.values

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]])